# 2章 Python数理最適化チュートリアル

### 2.1 連立一次方程式をPythonの数理最適化ライブラリで解く

In [1]:
"""
120x + 150y = 1440
x + y = 10
の単純な線形1次連立方程式を解くプログラム
"""

import pulp

problem = pulp.LpProblem('SLE', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print(f"Status:{pulp.LpStatus[status]}")
print(f"x = {x.value()}, y = {y.value()}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/99bddab5212b46eb8ccf5d71979adb29-pulp.mps max branch printingOptions all solution /tmp/99bddab5212b46eb8ccf5d71979adb29-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 13 RHS
At line 16 BOUNDS
At line 20 ENDATA
Problem MODEL has 2 rows, 3 columns and 4 elements
Coin0008I MODEL read with 0 errors
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status:Optimal
x = 2.0, y = 8.0


### 2.2 線形計画問題をPythonの数理最適化ライブラリで解く

解くべき最適化問題
   - max $n_p + 2n_q$
   - s.t.
      - $n_p, n_q >= 0$
      - $n_p+3n_q <= 30$
      - $2n_p+n_q <= 40$

In [8]:
"""
次のような線形計画問題を解くプログラム
   - max $n_p + 2n_q$
   - s.t.
      - $n_p, n_q >= 0$
      - $n_p+3n_q <= 30$
      - $2n_p+n_q <= 40$
"""

import pulp

problem = pulp.LpProblem("LP", pulp.LpMaximize)

np = pulp.LpVariable("n_p", cat="Continuous")
nq = pulp.LpVariable("n_q", cat="Continuous")

#制約条件の追加
problem += 1*np + 3*nq <= 30
problem += 2*np + 1*nq <= 40
problem += np >= 0
problem += nq >= 0

#目的関数
problem += np + 2*nq
#これらの書き方でも良い
# problem.setObjective(np + 2*nq)
# problem.objective = np + 2*nq

status = problem.solve()

print(f"status:{pulp.LpStatus[status]}")
print(f"n_p = {np.value()}, n_q = {nq.value()}, objective = {problem.objective.value()}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/36b18c48c70c4825a972990bd5201ed6-pulp.mps max branch printingOptions all solution /tmp/36b18c48c70c4825a972990bd5201ed6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 18 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj -0 Dual inf 2.999998 (2)
2  Obj 26
Optimal - objective value 26
After Postsolve, objective 26, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 26 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status:Optimal
n_p = 18.0, n_q = 4.0, object

#### 最適化問題の種類
この辺は詳細な書籍を読んだ方が正確な知識は得られそう。
  - 線形計画問題：変数が実数値をとる問題
  - 整数計画問題：変数が整数値のみを取る問題
  - 混合整数計画問題：変数が整数値を取るものと実数値を取るものがある問題
  - 0-1整数計画問題：変数が0か1で表せる問題
  - 凸二次計画問題：目的関数に凸な二次関数（係数行列が半正値対称行列）が現れる問題

### 2.3 規模の大きな数理最適化問題をPythonライブラリを使って解く

In [9]:
import pandas as pd
import pulp

##### 在庫データの確認

In [10]:
stock_df = pd.read_csv("./data/stocks.csv")
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


##### 製品の生産に必要な原料の量データの確認

In [12]:
require_df = pd.read_csv("./data/requires.csv")
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


##### 製品を生産した場合の利得データの確認

In [13]:
gain_df = pd.read_csv("./data/gains.csv")
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [15]:
Product_list = gain_df["p"].tolist()
Product_list

['p1', 'p2', 'p3', 'p4']

In [16]:
Material_list = stock_df["m"].tolist()
Material_list

['m1', 'm2', 'm3']